# 0. Import/Classes/Logger

In [18]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException, ElementNotInteractableException
import re
from time import sleep as time_sleep, time
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED
from selenium import webdriver
import requests
from webdriver_manager.chrome import ChromeDriverManager
import lxml
from itertools import chain
from typing import NamedTuple
import pickle
from collections import deque
from tqdm import tqdm
import pandas as pd
import numpy as np
import logging

In [9]:
# получение пользовательского логгера и установка уровня логирования
short_logger = logging.getLogger('short_logger')
short_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
short_handler = logging.FileHandler("data/short_logger.log", encoding='utf-8', mode='a')
short_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
short_handler.setFormatter(short_formatter)
# добавление обработчика к логгеру
short_logger.addHandler(short_handler)

# получение пользовательского логгера и установка уровня логирования
long_logger = logging.getLogger('long_logger')
long_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
long_handler = logging.FileHandler("data/long_logger.log", encoding='utf-8', mode='w')
long_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
long_handler.setFormatter(long_formatter)
# добавление обработчика к логгеру
long_logger.addHandler(long_handler)

In [10]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    alcohol: bool
    href: str
    page_href: str

class MyTimeoutErrore(Exception):
    def __init__(self, text):
        self.txt = text

In [4]:
driver = webdriver.Chrome()

# 1. Получение адресов городов

# 2. Получение ссылок на Магазины

## 2.1. Получение ссылки на старицы каталогов с магазинами

### 2.1.1. Функции

In [5]:
def _get_page_count(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            print(ex)
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            print(ex)
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise Exception("timeout")
    
    return out_list

### 2.1.2. Выполнение

In [6]:
# driver.get('https://edadeal.ru/lipeck/retailers')
url = 'https://edadeal.ru/lipeck/retailers'

In [7]:
temp_url_list = _get_page_count(driver, url, '?page=')
temp_url_list

['https://edadeal.ru/lipeck/retailers?page=1',
 'https://edadeal.ru/lipeck/retailers?page=2',
 'https://edadeal.ru/lipeck/retailers?page=3',
 'https://edadeal.ru/lipeck/retailers?page=4',
 'https://edadeal.ru/lipeck/retailers?page=5']

## 2.2. Получение непосредственно все ссылки на Магазины

### 2.2.1. Функции

In [8]:
def _get_retailers_links(driver: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[market_link]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1
        )

    
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
        except Exception as ex:
            print(ex)
            continue
        
        try:    
            for item in f1:
                href = host + item.find('a').get('href')
                title = item.find('a').find('h4').get('title')
                discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
                out_list.append(market_link(sity, title, discounts, False, href, None))
                
        except Exception as ex:
            print(ex)
            continue
        
        break
    
    else:
        raise MyTimeoutErrore('timeout')
    
    return out_list

### 2.2.2. Выполнение

In [9]:
temp_retailer_links = list()
for link in temp_url_list:
    temp_retailer_links.append(_get_retailers_links(driver, link, 'lipetsk'))

retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))
# print(retailer_links)

57


### 2.2.3. Save/Load

In [10]:
with open('data/retailer_links.pickle', 'wb') as file:
    pickle.dump(retailer_links, file)

In [11]:
with open('data/retailer_links.pickle', 'rb') as file:
    retailer_links = pickle.load(file)

# 3. Получение адресов Товаров

## 3.1. Получение ссылок на каталоги с товарами

### 3.1.1. Функции

In [12]:
def _get_page_count2(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> tuple[list[str],bool]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    try:
        driver.get(url)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    
    empty_page = False
    # Клик по кнопке "Есть 18 лет"
    alcohol = True
    # Проверка появления "Да, мне есть 18"
    try:
        button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
        button.click()
    except TimeoutException:
        alcohol = False
    except ElementNotInteractableException:
        alcohol = False
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            try:
                # Проверка на отсутствие скидок.
                driver.find_element(by=By.XPATH, value="//div[@class='b-dsk-stub__description']")
            except TimeoutException:
                raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
            else:
                empty_page = True
                out_list = None
                alcohol = True
                break
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol, empty_page)

def _get_all_links_to_market(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение адресов на все страницы с каталогами товаров
        Что бы после можно было использовать много поточность.
    """
    loger_fault = list()
    dead_line = 100
    # Очередь
    q = deque(list_url)

    out_list = list()
    with tqdm(total=len(list_url)) as pbar:
        while len(q) > 0:
            if dead_line < 0:
                break
            
            item = q.popleft()
            
            if item.discounts > 24:
                try:
                    temp_url_list, alcohol, empty_page = _get_page_count2(driver, item.href, '?page=', timeout)
                except MyTimeoutErrore as mte:
                    q.append(item)
                    loger_fault.append(mte)
                    dead_line -= 1
                    continue
                if not empty_page:
                    for href in temp_url_list:
                        out_list.append(market_link(item.sity,
                                                    item.title,
                                                    item.discounts,
                                                    alcohol,
                                                    item.href,
                                                    href))
                    
            elif item.discounts:
                out_list.append(market_link(item.sity,
                                            item.title,
                                            item.discounts,
                                            True,
                                            item.href,
                                            item.href))
            else:
                out_list.append(market_link(*item[:-1], None))
            
            pbar.update(1)

    return (out_list, loger_fault, q)

### 3.1.2. Выполнение

In [13]:
# Получение всех адресов на страницы с товарами.
retailer_links_pages, loger_fault, q = _get_all_links_to_market(driver, retailer_links, timeout=10)
len(retailer_links_pages)

100%|██████████| 57/57 [05:41<00:00,  5.99s/it]


914

### 3.1.3. Save/Load

In [14]:
with open('data/retailer_links_pages.pickle', 'wb') as file:
    pickle.dump(retailer_links_pages, file)

In [11]:
with open('data/retailer_links_pages.pickle', 'rb') as file:
    retailer_links_pages = pickle.load(file)

## 3.2. Получение информации о товарах и ссылки на них

### 3.2.1 Функции

In [17]:
def _get_page_count2(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    try:
        driver.get(url)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    alcohol = True
    # Проверка появления "Да, мне есть 18"
    try:
        button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
        button.click()
    except TimeoutException:
        alcohol = False
    except ElementNotInteractableException:
        alcohol = False
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='grid']"))
        except TimeoutException:
            raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"grid"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not GRID in url: {url}')
    
    return (out_list, alcohol)

def _get_all_links_to_market(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение адресов на все страницы с каталогами товаров
        Что бы после можно было использовать много поточность.
    """
    loger_fault = list()
    # Очередь
    q = deque(list_url)
    # Нужно вывести дейдлайн, который позволит вывести обработаные данные, и не обработанные. ???????????
    out_list = list()
    while len(q) > 0:
        item = q.popleft()
        
        if item.discounts > 24:
            try:
                temp_url_list, alcohol = _get_page_count2(driver, item.href, '?page=', timeout)
            except MyTimeoutErrore as mte:
                q.append(item)
                loger_fault.append(mte)
                continue
            
            for href in temp_url_list:
                out_list.append(market_link(item.sity,
                                            item.title,
                                            item.discounts,
                                            alcohol,
                                            item.href,
                                            href))
                
        elif item.discounts:
            out_list.append(market_link(item.sity,
                                        item.title,
                                        item.discounts,
                                        True,
                                        item.href,
                                        item.href))
        else:
            out_list.append(market_link(*item[:-1], None))

    return (out_list, loger_fault)



def _get_sku_info(driver: webdriver.Chrome, item: market_link, timeout: int=30):
    """
        Возвращает что-то об продукте.
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    
    # Вызов страницы
    try:
        driver.get(item.page_href)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    if item.alcohol:
        # Проверка появления "Да, мне есть 18"
        try:
            button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
            button.click()
        except TimeoutException:
            pass
        except ElementNotInteractableException:
            pass                                                                                # Нужно добавить логирование
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol)
    

### 3.2.2. Выполнение

#### 3.2.2.1. Сохранение всех страниц, для того что бы отточить работу парсера.

In [18]:
def _get_page(driver: webdriver.Chrome, item: market_link, timeout: int=30):
    """
        сохраняем страницы
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1
        )
    
    # Вызов страницы
    try:
        driver.get(item.page_href)
    
    except TimeoutException as ex:
        short_logger.warning(f"don't get response: {item.title} :: {item.page_href}")
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    except Exception as ex:
        short_logger.critical(f"other exception, {ex}: {item.title} :: {item.page_href}")
        long_logger.critical(f"other exception, {ex}: {item.title} :: {item.page_href}", exc_info=True)
    
    # Проверка сайта на 18 лет
    # Мы ранее собирали информацию, и приблизительно занем есть ли в данном магазине такая проверка или нет.
    if item.alcohol:
        try:
            # Ожидаем появляение кнопки, если она не повится, то ничего не делаем.
            button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
            button.click()
        
        except TimeoutException:
            short_logger.info(f'alcochol timeout: {item.title} :: {item.page_href}')
        except ElementNotInteractableException:
            short_logger.warning(f'alcochol not button: {item.title} :: {item.page_href}')
        except Exception as ex:
            short_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}')
            long_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}', exc_info=True)
    
    # Ручной тайм аут на выполнение всего процесса
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout*2:
        
        # Задержка действий до загрузки отслеживаемых елементов.
        # Варант провекрки загрузки страницы, проверка что текст не пустой
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@data-test-ref='grid']"))
        except TimeoutException:
            try:
                driver.find_element(by=By.XPATH, value="//div[@class='b-dsk-stub__description']")
            except TimeoutException:
                short_logger.warning(f"not table, timeout: {item.title} :: {item.page_href}")
                raise MyTimeoutErrore()
            else:
                short_logger.info(f"not page, timeout: {item.title} :: {item.page_href}")
                page = None
                break
            
        # Проверка наличия названия продукта в ячейке
        page = BS(driver.page_source, 'lxml')
        
        try:
            # Дожидаемся когда загрузится информация о первом товаре на странице
            f1 = page.find('div', {'data-test-ref':'grid'})
            f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
            f2[0].find('div', class_='b-srch-card__title').get_text(strip=True)
        except IndexError as ex:
            continue
        except Exception as ex:
            short_logger.critical(f"checking the product name, other exception, {ex}: {item.title} :: {item.page_href}")
            long_logger.critical(f"checking the product name, other exception, {ex}: {item.title} :: {item.page_href}", exc_info=True)

        break
    
    else:
        short_logger.warning(f"not table, timeout ALL: {item.title} :: {item.page_href}")
        raise MyTimeoutErrore()
    
    return driver.page_source



def _get_pages(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=20, dead_line: int=20) -> tuple[list[market_link], list[market_link]]:
    """
        Получение страниц
    """
    
    with tqdm(total=len(list_url)) as pbar:     # Прогресс бар
        
        q = deque(list_url)                     # Очередь
        out_list = list()
        
        while len(q) > 0:
            if dead_line < 0:                   # Ограничение количества попыток получить данные.
                return (out_list, q)
            
            item = q.popleft()
            if item.discounts:
                try:
                    page = _get_page(driver, item, timeout)
                except MyTimeoutErrore:
                    q.append(item)
                    dead_line -= 1
                    continue
                
                out_list.append(page)
            pbar.update(1)
            
    return (out_list, list(q))                        # Возвращаем отработаный лист и очередь(пустую или нет)

In [19]:
pages, other = _get_pages(driver, retailer_links_pages)

100%|██████████| 914/914 [35:32<00:00,  2.33s/it]  


In [20]:
# Сохранение страниц  в Zip файл.
with ZipFile('data/pages.zip', 'w', compression=ZIP_DEFLATED, compresslevel=1) as zip_file:
    for index, page in enumerate(pages):
        if page:
            zip_file.writestr(f'{index:05d}.html', page.encode())

In [12]:
# Поллучение страниц из Zip файла.
pages = list()
with ZipFile('data/pages.zip', 'r') as zf:
    for item in zf.filelist:
        with zf.open(item.filename) as file:
            pages.append(file.read())

In [13]:
url = retailer_links_pages[0].page_href


In [31]:
def _parser_sku_01(driver: webdriver.Chrome, market: market_link, date: datetime) -> pd.DataFrame:
    # Парсинг данных из файлов
    host = '/'.join(market.page_href.split('/', maxsplit=3)[:3])
    # 
    page = BS(driver, 'lxml')
    # page = driver
    f1 = page.find('div', {'data-test-ref':'grid'})
    f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
    
    test = list()
    
    for f3 in f2:
        # название
        title = f3.find('div', class_='b-srch-card__title').get_text(strip=True)
        
        # дата до истечения
        try:
            date_to = datetime.strptime(f3.find('time').get('datetime'), '%Y-%m-%d')
        except AttributeError:
            date_to = np.nan
        
        # Цена (разбита на части)
        b = (f3.find('div', class_='b-money__root-wrapper')
             .find('span', {'data-test-ref':"money-base"})
             .get_text(strip=True)
             .replace('\xa0', ''))
        try:
            s = (f3.find('div', class_='b-money__root-wrapper')
                .find('span', {'data-test-ref':"money-subunit"})
                .get_text(strip=True)
                .replace('\xa0', ''))
        except AttributeError:
            s = '0'
        price = float(f'{b}.{s}')
        
        # текущая валюта
        try:
            unit = f3.find('div', class_='b-money__root-wrapper').find('span', {'data-test-ref':"currency"}).get_text(strip=True)
        except AttributeError:
            unit = np.nan
            
        # скидка
        try:
            sale = f3.find('div', class_='b-srch-card__discount-label').get_text(strip=True)
        except AttributeError:
            sale = np.nan
        
        # вес
        try:
            f4 = (f3
                .find('div', class_='b-srch-card__info')
                .find('span')
                .get_text(strip=True)
                .replace('\xa0', ' ')
                .split('•'))
            vol = f4[0].strip()
            # руб/кг. руб/шт.
            vol_vs = f4[1].strip()
        except AttributeError:
            vol = np.nan
            vol_vs = np.nan
        # ссылка
        href = host + f3.find('a', class_="b-srch-card__card-wrapper").get('href')
        
        test.append((date, market.title, title, date_to, price, unit, sale, vol, vol_vs, href))

    return pd.DataFrame(test, columns=['date', 'market', 'title', 'date_to', 'price', 'unit', 'sale', 'vol', 'vol_vs', 'href'])

dataframe_list = list()

for page in pages:
    dataframe_list.append(_parser_sku_01(page, retailer_links_pages[0], datetime.strptime("2023-08-28", '%Y-%m-%d')))
out_df = pd.concat(dataframe_list, ignore_index=True)

In [33]:
with pd.option_context("display.max_columns", 500, 'display.max_colwidth', 200):
        
    display(out_df[out_df['vol'].isnull()])
    display(out_df[out_df['vol'].isnull()]['href']) 
out_df.info()

,date,market,title,date_to,price,unit,sale,vol,vol_vs,href
55,2023-08-28,Пятёрочка,Набор ножей Royal Kuchen сантоку и разделочный,2023-08-29 00:00:00,999.99,₽,–29%,NaN,NaN,https://edadeal.ru/lipeck/metaoffers/5dfec455-e3e7-5dab-b444-f2706d06761f
62,2023-08-28,Пятёрочка,Креветки варено-мороженные,2023-08-29 00:00:00,599.99,₽,–30%,NaN,NaN,https://edadeal.ru/lipeck/metaoffers/e543bf6a-7d29-5c1c-8165-d53f4243d8ae
64,2023-08-28,Пятёрочка,Щетка зубная Colgate Сенсация Свежести Клиниг тип средний,2023-09-05 00:00:00,149.99,₽,–29%,NaN,NaN,https://edadeal.ru/lipeck/metaoffers/6e4301f7-4752-5a98-bbaf-2d01eb8751c9
92,2023-08-28,Пятёрочка,Конфеты Левушка с карамельной начинкой,2023-09-05 00:00:00,399.90,₽,–26%,NaN,NaN,https://edadeal.ru/lipeck/metaoffers/772ba900-c969-56f9-9c9b-9f269745d882
93,2023-08-28,Пятёрочка,Колбаса Микояновский МК ливерная традиционная 400н,2023-09-05 00:00:00,64.99,₽,–28%,NaN,NaN,https://edadeal.ru/lipeck/metaoffers/4fb4424c-6f4a-5a1e-9c71-c61f827d4ad2
...,...,...,...,...,...,...,...,...,...,...
18567,2023-08-28,Пятёрочка,Крем насыщенный увлажняющий MASDAR для сильно обезвоженной кожи с гиалуроновой кислотой Zeitun,NaN,1544.00,₽,NaN,NaN,NaN,https://edadeal.ru/lipeck/offers/b5fc02af-30a1-5034-8e30-e6fbca2a89a8
18568,2023-08-28,Пятёрочка,Сыворотка для контура глаз Zeitun DARA против отеков и первых морщин Zeitun,NaN,1360.00,₽,NaN,NaN,NaN,https://edadeal.ru/lipeck/offers/d48b8df5-5082-5134-ab24-a951273350b4
18569,2023-08-28,Пятёрочка,Маска для лица ночная восстанавливающая против усталост с гиалуроновой кислотой Afterparty Beautific,NaN,1185.00,₽,NaN,NaN,NaN,https://edadeal.ru/lipeck/offers/8278d806-6736-5778-931b-3cc23ad74a5b
18572,2023-08-28,Пятёрочка,"Дезодорант-антиперспирант минеральный для мужчин нейтральный без запаха ""Ультра чистота"" Zeitun",NaN,536.00,₽,NaN,NaN,NaN,https://edadeal.ru/lipeck/offers/5b8e6caa-03d1-5bd3-9bf7-5a825c64ef1e


55       https://edadeal.ru/lipeck/metaoffers/5dfec455-e3e7-5dab-b444-f2706d06761f
62       https://edadeal.ru/lipeck/metaoffers/e543bf6a-7d29-5c1c-8165-d53f4243d8ae
64       https://edadeal.ru/lipeck/metaoffers/6e4301f7-4752-5a98-bbaf-2d01eb8751c9
92       https://edadeal.ru/lipeck/metaoffers/772ba900-c969-56f9-9c9b-9f269745d882
93       https://edadeal.ru/lipeck/metaoffers/4fb4424c-6f4a-5a1e-9c71-c61f827d4ad2
                                           ...                                    
18567        https://edadeal.ru/lipeck/offers/b5fc02af-30a1-5034-8e30-e6fbca2a89a8
18568        https://edadeal.ru/lipeck/offers/d48b8df5-5082-5134-ab24-a951273350b4
18569        https://edadeal.ru/lipeck/offers/8278d806-6736-5778-931b-3cc23ad74a5b
18572        https://edadeal.ru/lipeck/offers/5b8e6caa-03d1-5bd3-9bf7-5a825c64ef1e
18573        https://edadeal.ru/lipeck/offers/0fa11836-91d9-50ca-b6af-bbc73892fbd0
Name: href, Length: 11725, dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18579 entries, 0 to 18578
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     18579 non-null  datetime64[ns]
 1   market   18579 non-null  object        
 2   title    18579 non-null  object        
 3   date_to  6567 non-null   object        
 4   price    18579 non-null  float64       
 5   unit     18579 non-null  object        
 6   sale     13620 non-null  object        
 7   vol      6854 non-null   object        
 8   vol_vs   6854 non-null   object        
 9   href     18579 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 1.4+ MB
